In [ ]:
import gdown

file_id = '1Y2cTYR_t_10NAbznspE5bBjuATPdTgtq'
download_url = f'https://drive.google.com/uc?id={file_id}'
gdown.download(download_url, 'file.zip', quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1Y2cTYR_t_10NAbznspE5bBjuATPdTgtq
From (redirected): https://drive.google.com/uc?id=1Y2cTYR_t_10NAbznspE5bBjuATPdTgtq&confirm=t&uuid=1fd25d11-f83e-46c9-97fa-daaeb324729e
To: /content/file.zip
100%|██████████| 134M/134M [00:04<00:00, 30.5MB/s]


'file.zip'

In [ ]:
import zipfile

with zipfile.ZipFile('file.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/data')

In [ ]:
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from collections import Counter

data_dir = 'data/AISContest_Data'

data = []
labels = []

for class_label in os.listdir(data_dir):
    class_folder = os.path.join(data_dir, class_label)

    if os.path.isdir(class_folder):
        class_count = 0

        for file_name in os.listdir(class_folder):
            if file_name.endswith('.npy'):
                file_path = os.path.join(class_folder, file_name)
                class_data = np.load(file_path)

                data.append(class_data)
                labels.append(class_label)

                class_count += 1

        print(f'Loaded {class_count} samples from class: {class_label}')

data = np.array(data, dtype=np.float64)
labels = np.array(labels, dtype=np.int64)

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

class_counts = Counter(encoded_labels)
print("Original class distribution:", class_counts)

Loaded 1755 samples from class: 2
Loaded 497 samples from class: 3
Loaded 237 samples from class: 1
Loaded 418 samples from class: 4
Loaded 2906 samples from class: 0
Original class distribution: Counter({0: 2906, 2: 1755, 3: 497, 4: 418, 1: 237})
Maximum samples in any class: 1000


In [ ]:
balanced_data = []
balanced_labels = []

target_samples = 1000

for class_label in np.unique(encoded_labels):
    class_indices = np.where(encoded_labels == class_label)[0]

    num_samples = len(class_indices)

    if num_samples > target_samples:
        selected_indices = np.random.choice(class_indices, target_samples, replace=False)
    else:
        selected_indices = np.random.choice(class_indices, target_samples, replace=True)

    balanced_data.append(data[selected_indices])
    balanced_labels.append(encoded_labels[selected_indices])

balanced_data = np.concatenate(balanced_data)
balanced_labels = np.concatenate(balanced_labels)

new_class_counts = Counter(balanced_labels)
print(new_class_counts)


Counter({0: 1000, 1: 1000, 2: 1000, 3: 1000, 4: 1000})


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


test_size = 0.23
val_size = 0.25

x_train, x_test, y_train, y_test = train_test_split(balanced_data, balanced_labels, test_size=test_size, random_state=42)

x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=val_size / (1 - test_size), random_state=42)

num_samples, height, width, _ = x_train.shape
x_train_reshaped = x_train.reshape(num_samples, height * width)
scaler = MinMaxScaler()

x_train_normalized = scaler.fit_transform(x_train_reshaped)

x_val_reshaped = x_val.reshape(x_val.shape[0], height * width)
x_val_normalized = scaler.transform(x_val_reshaped)

x_test_reshaped = x_test.reshape(x_test.shape[0], height * width)
x_test_normalized = scaler.transform(x_test_reshaped)

x_train_normalized = x_train_normalized.reshape(x_train.shape)
x_val_normalized = x_val_normalized.reshape(x_val.shape)
x_test_normalized = x_test_normalized.reshape(x_test.shape)

print(f'Training set size: {x_train_normalized.shape}')
print(f'Validation set size: {x_val_normalized.shape}')
print(f'Test set size: {x_test_normalized.shape}')

Training set size: (3850, 51, 59, 1)
Validation set size: (374, 51, 59, 1)
Test set size: (776, 51, 59, 1)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout

model = models.Sequential()

model.add(Conv2D(32, kernel_size=3, input_shape=(51,59,1), kernel_initializer='glorot_uniform'))
model.add(Conv2D(32, kernel_size=3, padding='same', kernel_initializer='glorot_uniform'))
model.add(Conv2D(32, kernel_size=3, padding='same', kernel_initializer='glorot_uniform'))
model.add(Conv2D(32, kernel_size=3, padding='same', kernel_initializer='glorot_uniform'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format='channels_last'))
model.add(Conv2D(64, kernel_size=3, padding='same', kernel_initializer='glorot_uniform'))
model.add(Conv2D(64, kernel_size=3, padding='same', kernel_initializer='glorot_uniform'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format='channels_last'))
model.add(Conv2D(128, kernel_size=3, padding='same', kernel_initializer='glorot_uniform'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format='channels_last'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(512))
model.add(Dense(len(set(labels)), activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [54]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

import tensorflow as tf
tf.keras.backend.clear_session()

In [55]:

from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=len(set(labels)))
y_val = to_categorical(y_val, num_classes=len(set(labels)))

# Training the model
history = model.fit(x_train_normalized, y_train,
                    validation_data=(x_val_normalized, y_val),
                    epochs=15,
                    batch_size=16)

Epoch 1/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 101s 3s/step - accuracy: 0.2054 - loss: 1.6272 - val_accuracy: 0.2299 - val_loss: 1.6667
Epoch 2/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 143s 3s/step - accuracy: 0.2361 - loss: 1.6105 - val_accuracy: 0.2273 - val_loss: 1.6127
Epoch 3/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 141s 3s/step - accuracy: 0.2859 - loss: 1.5422 - val_accuracy: 0.2406 - val_loss: 1.6271
Epoch 4/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 98s 3s/step - accuracy: 0.3870 - loss: 1.3948 - val_accuracy: 0.4679 - val_loss: 1.4515
Epoch 5/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 94s 3s/step - accuracy: 0.5426 - loss: 1.1352 - val_accuracy: 0.5160 - val_loss: 1.4998
Epoch 6/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 145s 3s/step - accuracy: 0.6399 - loss: 0.9283 - val_accuracy: 0.5909 - val_loss: 1.4353
Epoch 7/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 142s 3s/step - accuracy: 0.7489 - loss: 0.6668 - val_accuracy: 0.6203 - val_loss: 1.6668
Epoch 8/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 143s 3s/step - accuracy: 0.8352 - loss: 0.4593 - val_accuracy: 0.5909 - val

In [56]:
from sklearn.metrics import f1_score


y_pred = model.predict(x_test_normalized)
y_pred_classes = np.argmax(y_pred, axis=1)

f1_micro = f1_score(y_test, y_pred_classes, average='micro')
print(f'F1 Micro Score: {f1_micro:.4f}')

25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 169ms/step
F1 Micro Score: 0.6843
